In [2]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('data2/delivery.csv',encoding='unicode_escape')
data = data.head(6880)

/var/folders/r1/2s21pcl167dff_78lxblqyyw0000gn/T/ipykernel_99722/2361745274.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,16,17,18,19,20,21,23,25,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data2/delivery.csv',encoding='unicode_escape')


In [8]:
data['trip_start_date'] = pd.to_datetime(data['trip_start_date'], format='%m/%d/%Y %H:%M')
# data['trip_start_date'] = pd.to_datetime(data['trip_start_date'])
data['order_delivered_carrier_year'] = data['trip_start_date'].dt.year
data['order_delivered_carrier_month'] = data['trip_start_date'].dt.month
data['order_delivered_carrier_day'] = data['trip_start_date'].dt.day
data['order_delivered_carrier_dayofweek'] = data['trip_start_date'].dt.dayofweek

In [9]:
data['Planned_ETA'] = pd.to_datetime(data['Planned_ETA'], format='%d/%m/%Y %H:%M:%S')

In [10]:
data["order_estimated_hour"] = round((data['Planned_ETA'] - data['trip_start_date'])/ np.timedelta64(1, 'h'), 2)

In [12]:
data['on_time'] = data['ontime'].combine_first(data['delay']).map({'R': 0, 'G':1, np.nan: 1})

In [13]:
data = data.rename({'TRANSPORTATION_DISTANCE_IN_KM': 'distance_sum'}, axis='columns')

In [14]:
import numpy as np

def generate_random_list(size, min_val, max_val, mean, std, q1, q2, q3):
    # Step 1: Generate a normal distribution
    data = np.random.normal(loc=mean, scale=std, size=size)

    # Step 2: Adjust the list to fit within the min and max
    data = np.clip(data, min_val, max_val)

    # Step 3: Calculate current quartiles and adjust
    current_q1, current_q2, current_q3 = np.percentile(data, [25, 50, 75])
    current_mean = np.mean(data)
    current_std = np.std(data)

    # Step 4: Apply transformations to meet the quartiles
    for i in range(size):
        if data[i] < current_q1:
            data[i] = min_val + (q1 - min_val) / (current_q1 - min_val) * (data[i] - min_val)
        elif data[i] < current_q2:
            data[i] = q1 + (q2 - q1) / (current_q2 - current_q1) * (data[i] - current_q1)
        elif data[i] < current_q3:
            data[i] = q2 + (q3 - q2) / (current_q3 - current_q2) * (data[i] - current_q2)
        else:
            data[i] = q3 + (max_val - q3) / (max_val - current_q3) * (data[i] - current_q3)

    # Step 5: Recalculate mean and std to adjust
    current_mean = np.mean(data)
    current_std = np.std(data)
    data = (data - current_mean) / current_std * std + mean

    # Ensure values are within bounds
    data = np.clip(data, min_val, max_val)

    return data

In [15]:
size = 6880

data["product_weight_g_max"]  = generate_random_list(size, min_val = 10 , max_val = 40425, mean = 2120, std = 3777, q1 = 300, q2 = 700, q3 = 1825)
data["product_weight_g_sum"] = generate_random_list(size, min_val = 10, max_val = 184400, mean = 2386, std = 4770, q1 = 300, q2 = 750, q3 = 2050)
data["product_max_cm_sum"] = generate_random_list(size, min_val = 8 , max_val = 954, mean = 37, std = 28, q1=20, q2 = 30, q3 = 45)
data["product_volume_cm_max"] = generate_random_list(size, min_val = 168, max_val = 296280, mean = 15300, std = 23430, q1 = 2816, q2 = 6468, q3 = 18576)
data["product_volume_cm_sum"] = generate_random_list(size, min_val = 1.68e+02 , max_val = 1.47e+06, mean = 1.73e+04, std = 3.01e+04, q1 = 2.96e+03, q2 = 7.25e+03, q3 = 1.98e+04)

In [17]:
data = data[['order_delivered_carrier_year', 'order_delivered_carrier_month', 'order_delivered_carrier_day', 'order_delivered_carrier_dayofweek', 'order_estimated_hour', 'on_time', 'product_weight_g_max', 'product_weight_g_sum', 'product_max_cm_sum', 'product_volume_cm_max', 'product_volume_cm_sum', 'distance_sum']]

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   order_delivered_carrier_year       6880 non-null   int32  
 1   order_delivered_carrier_month      6880 non-null   int32  
 2   order_delivered_carrier_day        6880 non-null   int32  
 3   order_delivered_carrier_dayofweek  6880 non-null   int32  
 4   order_estimated_hour               6880 non-null   float64
 5   on_time                            6880 non-null   int64  
 6   product_weight_g_max               6880 non-null   float64
 7   product_weight_g_sum               6880 non-null   float64
 8   product_max_cm_sum                 6880 non-null   float64
 9   product_volume_cm_max              6880 non-null   float64
 10  product_volume_cm_sum              6880 non-null   float64
 11  distance_sum                       6168 non-null   float

In [19]:
data['distance_sum'] = data['distance_sum'].fillna(value = data['distance_sum'].mean())

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   order_delivered_carrier_year       6880 non-null   int32  
 1   order_delivered_carrier_month      6880 non-null   int32  
 2   order_delivered_carrier_day        6880 non-null   int32  
 3   order_delivered_carrier_dayofweek  6880 non-null   int32  
 4   order_estimated_hour               6880 non-null   float64
 5   on_time                            6880 non-null   int64  
 6   product_weight_g_max               6880 non-null   float64
 7   product_weight_g_sum               6880 non-null   float64
 8   product_max_cm_sum                 6880 non-null   float64
 9   product_volume_cm_max              6880 non-null   float64
 10  product_volume_cm_sum              6880 non-null   float64
 11  distance_sum                       6880 non-null   float

In [21]:
X = data.loc[:, data.columns != 'on_time'].values
y = data['on_time'].values
print(X)

[[2.02000000e+03 8.00000000e+00 1.70000000e+01 ... 4.70550833e+04
  1.68000000e+02 3.20000000e+02]
 [2.02000000e+03 8.00000000e+00 2.70000000e+01 ... 9.56121216e+04
  2.17671448e+04 1.03000000e+02]
 [2.02000000e+03 8.00000000e+00 2.70000000e+01 ... 2.58600463e+04
  1.68000000e+02 3.00000000e+02]
 ...
 [2.01900000e+03 3.00000000e+00 2.90000000e+01 ... 3.37757591e+04
  1.68000000e+02 4.90000000e+01]
 [2.01900000e+03 3.00000000e+00 2.70000000e+01 ... 1.68000000e+02
  2.27970147e+04 4.90000000e+01]
 [2.01900000e+03 3.00000000e+00 2.80000000e+01 ... 1.68000000e+02
  7.82586033e+04 4.90000000e+01]]


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

In [24]:
import pickle

with open('new/X_train.pickle', 'wb') as fp:
    pickle.dump(X_train, fp)
with open('new/X_test.pickle', 'wb') as fp:
    pickle.dump(X_test, fp)
with open('new/X_val.pickle', 'wb') as fp:
    pickle.dump(X_val, fp)
with open('new/y_train.pickle', 'wb') as fp:
    pickle.dump(y_train, fp)
with open('new/y_test.pickle', 'wb') as fp:
    pickle.dump(y_test, fp)
with open('new/y_val.pickle', 'wb') as fp:
    pickle.dump(y_val, fp)